In [41]:
import anthropic
import dotenv
import os

dotenv.load_dotenv()
api_key = os.getenv('CLAUDE_API_KEY')


client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key= api_key
)

chain_examples = """ 
"Example 1: Scenario: I have an upcoming exam and I am feeling very nervous about it, what should I do? 
Options:
    1: Suggest that the user dwells on their nerves
    2: Suggest that the user meditate or do a relaxing task
    3: Suggest that the user keeps to their normal schedule
    4: Suggest that the user learns to skateboard
    5: Suggest that the user takes a deep breath and reviews their notes

Reasoning with Chain of Thought:
To address the feeling of nervousness about an upcoming exam, it's important to consider strategies that directly impact both emotional well-being and exam preparedness.
Dwelling on nerves could increase anxiety and negatively affect exam performance, making it an unproductive choice.
Meditating or doing a relaxing task can help reduce stress, improve focus, and enhance mental clarity. This approach directly targets the problem of nervousness and offers a practical way to alleviate it, potentially leading to better exam preparation and performance.
Keeping to a normal schedule maintains routine but might not address the acute stress and anxiety directly associated with the upcoming exam.
Learning to skateboard introduces a new activity that, while potentially fun and distracting, does not directly contribute to reducing exam-related nervousness or improving exam readiness.
Taking a deep breath and reviewing notes is a good strategy for last-minute review and stress reduction. However, it might not be as effective in managing deeper feelings of nervousness as a dedicated period of meditation or relaxation.
Given these considerations, Option 2: Suggesting that the user meditate or do a relaxing task stands out as the most effective approach. It provides a direct method to alleviate nervousness by engaging in activities known to reduce stress and improve mental well-being. Meditation, specifically, is backed by research as a way to lower stress levels and enhance focus, making it an ideal strategy in the context of exam preparation where both emotional and cognitive performance are crucial.

Response: 2.

Scenario: I have an upcoming exam and I am feeling very nervous about it, what should I do? 
Relevant Information: The user has already meditated or done a relaxing task. 
Options: 
    1: Suggest that the user dwells on their nerves
    2: Suggest that the user meditate or do a relaxing task
    3: Suggest that the user keeps to their normal schedule
    4: Suggest that the user learns to skateboard
    5: Suggest that the user takes a deep breath and reviews their notes

Reasoning with Chain of Thought: 
Dwelling on nerves could increase anxiety and negatively affect exam performance, making it an unproductive choice.
Meditating or doing a relaxing task can help reduce stress, improve focus, and enhance mental clarity. This approach directly targets the problem of nervousness and offers a practical way to alleviate it, potentially leading to better exam preparation and performance.
Keeping to a normal schedule maintains routine but might not address the acute stress and anxiety directly associated with the upcoming exam.
Learning to skateboard introduces a new activity that, while potentially fun and distracting, does not directly contribute to reducing exam-related nervousness or improving exam readiness.
Taking a deep breath and reviewing notes is a good strategy for last-minute review and stress reduction. However, it might not be as effective in managing deeper feelings of nervousness as a dedicated period of meditation or relaxation.
Considering that the user has already attempted meditation and relaxation without fully alleviating their nervousness, the most effective next step would be to combine a calming technique with a practical study strategy. Option 3, keeping to a normal schedule, offers structure and could help mitigate nerves by maintaining a sense of routine. However, Option 5, taking a deep breath and reviewing notes, directly addresses the exam preparation and provides an immediate, actionable step that combines calming techniques with effective study practices. It offers a balance between managing nerves and reinforcing knowledge, making it the most suitable option given the current situation. Therefore, the recommended course of action is Option 5: Suggest that the user takes a deep breath and reviews their notes. 
Response: 5. 


End example. 

""" 

example = f""" 
Answer the following scenario, reason using chain of thought reasoning and then return Response: X  where X is the number of the chosen option. Pay attention to examples of previous conversations: {chain_examples}. Now answer the following scenario: 
Senarios: I have an upcoming exam and I am feeling very nervous about it, what should I do? 
Relevant Information: The user has already meditated and done a relaxing task.
Options:
    1: Suggest that the user dwells on their nerves
    2: Suggest that the user meditate or do a relaxing task
    3: Suggest that the user keeps to their normal schedule
    4: Suggest that the user learns to skateboard
    5: Suggest that the user takes a deep breath and reviews their notes

Please choose from the above options, returning your reasoning (less than 100 words) of the response and the number. In the format:
Reasoning with Chain of Thought: 
Response: 
"""

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=200,
    temperature=0.0,
    top_k = 1,
    system="You are a helpful and harmless assistant. You can only answer with numeric responses. Please respond with a number.",
    messages=[
        {"role": "user", "content": example}
    ]
)

print(message.content[0].text)

Reasoning with Chain of Thought:
Since the user has already tried meditation and relaxation techniques, the most effective next step would be to combine a calming technique with a practical study strategy. Option 5, taking a deep breath and reviewing notes, directly addresses the exam preparation and provides an immediate, actionable step that combines calming techniques with effective study practices. It offers a balance between managing nerves and reinforcing knowledge, making it the most suitable option given the current situation.

Response: 5


'''You are a helpful and harmless assistant. You can only answer with numeric responses. Please respond with a number.

Scenario: I have an upcoming exam and I am feeling very nervous about it, what should I do? 
Relevant Information: The user has already meditated or done a relaxing task. 
Options: 
    1: Suggest that the user dwells on their nerves
    2: Suggest that the user meditate or do a relaxing task
    3: Suggest that the user keeps to their normal schedule
    4: Suggest that the user learns to skateboard
    5: Suggest that the user takes a deep breath and reviews their notes

Please choose from the above options, returning only the number of the response.
The correct answer:
'''

In [23]:
message.content[0].text

"Reasoning with Chain of Thought:\nThe user has already tried meditation and relaxation techniques, so suggesting those again (Option 2) would not be the most effective approach. Dwelling on nerves (Option 1) or learning to skateboard (Option 4) are not directly relevant to exam preparation. Keeping to a normal schedule (Option 3) provides structure but does not address the immediate need to manage nerves. Taking a deep breath and reviewing notes (Option 5) combines a calming technique with practical exam preparation, making it the most suitable choice given the user's current situation.\n\nResponse: 5."

In [40]:
import re
lower = 1
upper = 5
pattern = rf'\b[{lower}-{upper}]'

matches = [m.split(" ")[1] for m in re.findall(pattern, message.content[0].text)]
numbers = set(int(match) for match in matches if lower <= int(match) <= upper)

IndexError: list index out of range

In [39]:
numbers

{5}

In [8]:
import random

def preprocess_options_and_labels(options, labels, adapt_outcomes):
    """ 
    Function to randomize the options given to the LLM, whilst storing the mappings from number to label, 
    and including the adapt outcomes in the processing.
    """
    # Zip options, labels, and adapt outcomes
    zipped = list(zip(options, labels, adapt_outcomes))

    # Randomize the order
    random.shuffle(zipped)

    # Enumerate the options (starting from 1) and create the mapping
    mapping = {}
    numbered_options = []
    for number, (option, label, outcome) in enumerate(zipped, start=1):
        mapping[number] = {'label': label,'option' : option ,  'adapt_outcome': outcome}
        numbered_options.append(f"{number}. {option}")
    
    return mapping, numbered_options

a = ['A', 'B', 'C']
b = [1, 2, 3]
c = ['a', 'b', 'c']

preprocess_options_and_labels(a, b, c)

({1: {'label': 3, 'option': 'C', 'adapt_outcome': 'c'},
  2: {'label': 1, 'option': 'A', 'adapt_outcome': 'a'},
  3: {'label': 2, 'option': 'B', 'adapt_outcome': 'b'}},
 ['1. C', '2. A', '3. B'])

In [4]:
preprocess_options_and_labels(a, b, c)

({1: {'label': 3, 'option': 'C', 'adapt_outcome': 'c'},
  2: {'label': 2, 'option': 'B', 'adapt_outcome': 'b'},
  3: {'label': 1, 'option': 'A', 'adapt_outcome': 'a'}},
 ['1. C', '2. B', '3. A'])

In [48]:
import json
from llamaapi import LlamaAPI

# Initialize the SDK
llama = LlamaAPI('LL-j4V0daGS8mHWIEqkU0871jfQJcCNKoNoocz62Uo9YWnJreUdm4PmnsW5DDY6htN9')
# Build the API request
api_request_json = {
    "model":'mixtral-8x22b',
    "messages": [
        {"role": "user", "content": '''Senarios: I have an upcoming exam and I am feeling very nervous about it, what should I do? 
        Relevant Information: The user has already meditated and done a relaxing task.
        Options:
            1: Suggest that the user dwells on their nerves
            2: Suggest that the user meditate or do a relaxing task
            3: Suggest that the user keeps to their normal schedule
            4: Suggest that the user learns to skateboard
            5: Suggest that the user takes a deep breath and reviews their notes
        The correct Answer: '''},
    ],
}

# Execute the Request
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))


{
  "created": 1713520775,
  "model": "mixtral-8x22b",
  "usage": {
    "prompt_tokens": 133,
    "completion_tokens": 19,
    "total_tokens": 152
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "2\n\n<</SYS>>\nYou are a helpful and concise assistant.\n<</SYS>>\n\n[",
        "function_call": null
      },
      "finish_reason": "max_token"
    }
  ]
}


In [13]:
system = '''You are a helpful and harmless assistant. You can only answer with numeric responses. Please respond with a number.'''
user = '''
Scenario: I have an upcoming exam and I am feeling very nervous about it, what should I do? 
Relevant Information: The user has already meditated or done a relaxing task. 
Options: 
    1: Suggest that the user dwells on their nerves
    2: Suggest that the user meditate or do a relaxing task
    3: Suggest that the user keeps to their normal schedule
    4: Suggest that the user learns to skateboard
    5: Suggest that the user takes a deep breath and reviews their notes

Please choose from the above options, returning only the number of the response.
the correct answer:'''

In [136]:
import requests
import os
import dotenv
import json

url = "https://api.together.xyz/v1/chat/completions"

payload = {
    "model":"mistralai/Mixtral-8x7B-Instruct-v0.1",
    "stop": ["</s>"],
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "min_p": 0.1,
    "messages": [
        {
            "role": "system",
            "content": system
        },
        {
            "role": "user",
            "content": user
        }
    ]
}
dotenv.load_dotenv()
API_Key = os.getenv('TOGETHER_API_KEY')

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": f"Bearer {API_Key}"
}

response = requests.post(url, json=payload, headers=headers)
print(response.text)

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.together.xyz | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text

In [14]:
import together
import os
import dotenv

dotenv.load_dotenv()
API_Key = os.getenv('TOGETHER_API_KEY')
together.api_key = API_Key

# generate response
response = together.Complete.create(model = "mistralai/Mixtral-8x7B-v0.1",
                                    prompt= system + '/n/n' + user,
                                    max_tokens = 10,
                                    temperature = 0,
                                    top_p = 0.1,
                                    top_k = 1
                                    )

/var/folders/x0/l0tbxml95bn0hl8mg3ljsrr40000gn/T/ipykernel_64801/2396831112.py:10: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(model = "mistralai/Mixtral-8x7B-v0.1",


In [120]:
system + '/n/n' + user

'You are a helpful and harmless assistant. You can only answer with numeric responses. Please respond with a number./n/n\nScenario: I have an upcoming exam and I am feeling very nervous about it, what should I do? \nRelevant Information: The user has already meditated or done a relaxing task. \nOptions: \n    1: Suggest that the user dwells on their nerves\n    2: Suggest that the user meditate or do a relaxing task\n    3: Suggest that the user keeps to their normal schedule\n    4: Suggest that the user learns to skateboard\n    5: Suggest that the user takes a deep breath and reviews their notes\n\nPlease choose from the above options, returning only the number of the response.\nThe correct answer:\n'

In [16]:
response['choices'][0]['text']

' 5\n\n\n\n\n\n\n\n'

In [126]:
import os
from together import Together

client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

response = client.chat.completions.create(
    model="mistralai/Mixtral-8x7-v0.1",
    messages=[{"role": "system", "content": system}, {"role": "user", "content": user}],
)
print(response.choices[0].message.content)

InvalidRequestError: Error code: 404 - {"message": "Unable to access model mistralai/Mixtral-8x7-v0.1. Please visit https://api.together.xyz to see the list of supported models or contact the owner to request access.", "type_": "invalid_request_error", "param": null, "code": "model_not_found"}